In [83]:
from bs4 import BeautifulSoup
import pandas as pd
import string
import requests
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yanwarutsuksawat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/yanwarutsuksawat/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [84]:
def get_and_clean_data():
    data = pd.read_csv('data/software_developer_united_states_1971_20191023_1.csv')
    description = data['job_description']
    cleaned_description = description.apply(lambda s: s.translate(str.maketrans('','',
                                                                                string.punctuation + u'\xa0')))
    cleaned_description = cleaned_description.apply(lambda s: s.lower())
    cleaned_description = cleaned_description.apply(
        lambda s: s.translate(
            str.maketrans(string.whitespace, ' ' * len(string.whitespace), '')))
    cleaned_description = cleaned_description.drop_duplicates()
    return cleaned_description


def simple_tokenize(data):
    clean_description = data.apply(lambda s: [x.strip() for x in s.split()])
    return clean_description


def parse_job_description():
    clean_description = get_and_clean_data()
    clean_description = simple_tokenize(clean_description)
    return clean_description


def count_java():
    parse_description = parse_job_description()
    count_java = parse_description.apply(lambda s: 'java' in s).sum()
    print('java: ' + str(count_java) + ' of ' + str(
        parse_description.shape[0]))


def parse_db():
    html_doc = requests.get("https://db-engines.com/en/ranking").content
    soup = BeautifulSoup(html_doc, 'html.parser')
    db_table = soup.find("table", {"class": "dbi"})  # find the table that class="dbi"
    all_db = [''.join(s.find('a').findAll(string=True, recursive=False)).strip() for s in
              db_table.findAll("th", {
                  "class": "pad-l"})]
    all_db = list(dict.fromkeys(all_db))
    db_list = all_db[:10]
    db_list = [s.lower() for s in db_list]
    db_list = [[x.strip() for x in s.split()] for s in
               db_list]
    return db_list

In [85]:
cleaned_db = parse_db()
parse_description = parse_job_description()
raw = [None] * len(cleaned_db)
for i, db in enumerate(cleaned_db):
  raw[i] = parse_description.apply(lambda s: np.all([x in s for x in db])).sum()



In [86]:
with_java = [None] * len(cleaned_db)
for i, db in enumerate(cleaned_db):
    with_java[i] = parse_description.apply(lambda s: np.all([x in s for x in db]) and 'java' in s).sum()

In [87]:
for i, db in enumerate(cleaned_db):
    print(' '.join(db) + ' + java: ' + str(with_java[i]) + ' of ' + str(raw[i]) + ' (' +
          str(np.around(with_java[i] / raw[i] * 100, 2)) + '%)')

oracle + java: 913 of 1392 (65.59%)
mysql + java: 397 of 667 (59.52%)
microsoft sql server + java: 239 of 868 (27.53%)
postgresql + java: 161 of 261 (61.69%)
mongodb + java: 166 of 296 (56.08%)
redis + java: 40 of 106 (37.74%)
elasticsearch + java: 112 of 161 (69.57%)
ibm db2 + java: 33 of 48 (68.75%)
sqlite + java: 5 of 28 (17.86%)
microsoft access + java: 78 of 256 (30.47%)


1.) What DB should I learn after java?
  - Ans: elasticsearch or ibm db2

In [88]:
with_oracle = [None] * len(cleaned_db)
for i, db in enumerate(cleaned_db):
    with_oracle[i] = parse_description.apply(lambda s: np.all([x in s for x in db]) and 'oracle' in s).sum()

In [89]:
for i, db in enumerate(cleaned_db):
    print(' '.join(db) + ' + oracle: ' + str(with_oracle[i]) + ' of ' + str(raw[i]) + ' (' +
          str(np.around(with_oracle[i] / raw[i] * 100, 2)) + '%)')

oracle + oracle: 1392 of 1392 (100.0%)
mysql + oracle: 312 of 667 (46.78%)
microsoft sql server + oracle: 195 of 868 (22.47%)
postgresql + oracle: 100 of 261 (38.31%)
mongodb + oracle: 104 of 296 (35.14%)
redis + oracle: 12 of 106 (11.32%)
elasticsearch + oracle: 32 of 161 (19.88%)
ibm db2 + oracle: 23 of 48 (47.92%)
sqlite + oracle: 17 of 28 (60.71%)
microsoft access + oracle: 51 of 256 (19.92%)


2.) Which DB is in demand alongside oracle?
   - Ans: sqlite

In [120]:
lang = [['java'], ['python'], ['c'], ['kotlin'], ['swift'], ['rust'], ['ruby'], ['scala'], ['julia'], ['lua']]
parsed_description = parse_job_description()
parsed_db = parse_db()
all_terms = lang + parsed_db

query_map = pd.DataFrame(parsed_description.apply(lambda s: [1 if np.all([d in s for d in db])
                                                             else 0 for db in all_terms]).values.tolist(),
                         columns=[' '.join(d) for d in all_terms])


q1 = query_map[query_map['python'] > 0].apply(lambda s: np.where(s == 1)[0],
                                                    axis=1).apply(lambda s: list(query_map.columns[s]))
with_python = [None] * len(lang)
raw = [None] * len(lang)
for i, langs in enumerate(lang):
    with_python[i] = q1.apply(lambda s: np.all([x in s for x in langs]) and 'python' in s).sum()


java + python: 830 of 1379
python + python: 1379 of 1379
c + python: 689 of 1379
kotlin + python: 6 of 1379
swift + python: 37 of 1379
rust + python: 6 of 1379
ruby + python: 181 of 1379
scala + python: 76 of 1379
julia + python: 1 of 1379
lua + python: 11 of 1379
java + python: 830 of 1379 (60.19%)
python + python: 1379 of 1379 (100.0%)
c + python: 689 of 1379 (49.96%)
kotlin + python: 6 of 1379 (0.44%)
swift + python: 37 of 1379 (2.68%)
rust + python: 6 of 1379 (0.44%)
ruby + python: 181 of 1379 (13.13%)
scala + python: 76 of 1379 (5.51%)
julia + python: 1 of 1379 (0.07%)
lua + python: 11 of 1379 (0.8%)


In [121]:
for i, langs in enumerate(lang):
    print(' '.join(langs) + ' + python: ' + str(with_python[i]) + ' of ' + str(q1.shape[0]) + ' (' +
            str(np.around(with_python[i] / q1.shape[0] * 100, 2)) + '%)')

java + python: 830 of 1379 (60.19%)
python + python: 1379 of 1379 (100.0%)
c + python: 689 of 1379 (49.96%)
kotlin + python: 6 of 1379 (0.44%)
swift + python: 37 of 1379 (2.68%)
rust + python: 6 of 1379 (0.44%)
ruby + python: 181 of 1379 (13.13%)
scala + python: 76 of 1379 (5.51%)
julia + python: 1 of 1379 (0.07%)
lua + python: 11 of 1379 (0.8%)


3.) What programing language is in demand alongside python?
  - Ans: Java

4.) Create one question beginning with "WHY" that can be answered using the data we've analyzed during this class. Then, provide your solution to the question.
- Why do we need to learn Microsoft Sql Server after learning C?

In [131]:
cleaned_db = parse_db()
parse_description = parse_job_description()
raw = [None] * len(cleaned_db)
for i, db in enumerate(cleaned_db):
    raw[i] = parse_description.apply(lambda s: np.all([x in s for x in db])).sum()

with_c = [None] * len(cleaned_db)
for i, db in enumerate(cleaned_db):
    with_c[i] = parse_description.apply(lambda s: np.all([x in s for x in db]) and 'c' in s).sum()

for i, db in enumerate(cleaned_db):
    print(' '.join(db) + ' + c: ' + str(with_c[i]) + ' of ' + str(raw[i]) + ' (' +
              str(np.around(with_c[i] / raw[i] * 100, 2)) + '%)')

oracle + c: 511 of 1392 (36.71%)
mysql + c: 278 of 667 (41.68%)
microsoft sql server + c: 658 of 868 (75.81%)
postgresql + c: 92 of 261 (35.25%)
mongodb + c: 99 of 296 (33.45%)
redis + c: 36 of 106 (33.96%)
elasticsearch + c: 54 of 161 (33.54%)
ibm db2 + c: 13 of 48 (27.08%)
sqlite + c: 16 of 28 (57.14%)
microsoft access + c: 192 of 256 (75.0%)


Solution -> Based on the data. the percentage that Microsoft Sql Server appearing in the job description document is significantly high from other DBMS.